In [ ]:
import os
import logging
import sys

import openai
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer


load_dotenv()

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
)

openai.api_key = os.environ["OPENAI_API_KEY"]

space_name = "policies_aa"
edge_types, rel_prop_names = ["relationship"], [
    "name"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(
    persist_dir="./data/index/policies_aa",
    graph_store=graph_store
)

kg_index = load_index_from_storage(
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

chat_engine = kg_index.as_chat_engine(memory=chat_memory)

question = "What is the fee for a checked bag on American Airlines?"
response = chat_engine.chat(question)
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))

question_2 = "I'm flying economy class to Cuba."
response_2 = chat_engine.chat(question_2)
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))